In [6]:
#from utils import dump_dataset
import fasttext
import pandas as pd
import nltk
#nltk.download('stopwords') # Added (requested by system)
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 

from sklearn.model_selection import train_test_split

stop_words = stopwords.words('french')


df = pd.read_parquet('data/nt_data_fr.gzip')
toBeClassed = df

In [2]:
def preprocess(sentence, max_words=64):
    sentence = sentence.lower()
    
    tokenizer = nltk.RegexpTokenizer(r"\w+")
    word_tokens = tokenizer.tokenize(sentence)
  
    filtered_sentence = [w for w in word_tokens if not w in stop_words][:max_words]
  
    return " ".join(filtered_sentence)

In [3]:
def save_datafile(df, filename):
    with open(filename, 'w') as f:
        for each_text, each_label in zip(df['preproc'], df['category']):
            f.writelines(f'__label__{each_label} {each_text}\n')

In [4]:
def save_categories_in_one_file(df, filename, num_articles):
    with open(filename, 'w') as f:
        f.write('{"response":{')
        for each_category, index in zip(categories, range(len(categories))):
            f.write('"'+each_category + '":{"hits":[')
            label = "__label__" + each_category
            specific_df = df[df['category'].isin([label])]
            for each_row in range(min(num_articles, specific_df.shape[0])): 
                if(each_row == 0):
                    f.writelines(f'{specific_df.iloc[each_row].to_json()}')
                else:
                    f.writelines(f',{specific_df.iloc[each_row].to_json()}')
            if(index == len(categories)-1):
                f.write(']}')
            else:
                f.write(']},')
            
        f.write('}}')

In [7]:
categories = ['sciences','economie','sport','culture', 'sante', 'suisse', 'people', 'societe', 'insolite', 'politique', 'international', 'tech', 'faits-divers']

df['category'] = df['url'].apply(lambda x :  x.split('/')[3])
df = df[df['category'].isin(categories)]

df['preproc'] = (df['title'] +  " " + df['text']).apply(preprocess)
toBeClassed['preproc'] = (toBeClassed['title'] +  " " + toBeClassed['text']).apply(preprocess)

train, test = train_test_split(df, test_size=0.2)

save_datafile(train, 'data/fasttext.train')
save_datafile(test, 'data/fasttext.valid')

model = fasttext.train_supervised(input="data/fasttext.train", lr=1.0,epoch=50, dim=100)
model.test("data/fasttext.valid")

toBeClassed['category'] = toBeClassed['preproc'].apply(lambda preproc :  model.predict(preproc, k =1)[0][0])

save_categories_in_one_file(toBeClassed, "output/categories2.json", 10)

<ipython-input-7-aeaf2f6de0d0>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['preproc'] = (df['title'] +  " " + df['text']).apply(preprocess)
